In [ ]:
"""Part 1: MEV Mixture Model

Case Study 5
Demand Modeling: 1.202
Spring 2019
Alexandra Berke
"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR_mean = Beta('ASC_CAR_mean',-0.252,-10,10,0)
ASC_CAR_std = Beta('ASC_CAR_std',0.0926,-10,10,0)
ASC_SBB_mean = Beta('ASC_SBB_mean',0,-10,10,1) # fix to zero
ASC_SBB_std = Beta('ASC_SBB_std',0,-10,10,1) # fixed to zero
ASC_SM_mean = Beta('ASC_SM_mean',0.0239,-10,10,0)
ASC_SM_std = Beta('ASC_SM_std',0.0279,-10,10,0)

B_CAR_COST = Beta('B_CAR_COST',0,-10,10,0)
B_HE = Beta('B_HE',0,-10,10,0)
B_SM_COST = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR = Beta('B_SENIOR',0,-10,10,0)
B_GA = Beta('B_GA',0,-10,80,0)
B_FIRST = Beta('B_FIRST',0,-10,80,0)
# parameters relevant to the nests
MU_rail = Beta('MU_rail',1,1,10,0)

# Random parameters
# Normalization: First estimate all three, then fix the smallest std to 0 and estimate again
ASC_CAR_random = ASC_CAR_mean + ASC_CAR_std * bioDraws('ASC_CAR_random','NORMAL')
ASC_SBB_random = ASC_SBB_mean + ASC_SBB_std * bioDraws('ASC_SBB_random','NORMAL')
ASC_SM_random = ASC_SM_mean + ASC_SM_std * bioDraws('ASC_SM_random','NORMAL')

# Define here arithmetic expressions for name that are not directly 
# available from the data
SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

# Utilities
V_Car_SP = ASC_CAR_random * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
V_Train_SP = ASC_SBB_random * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
            + B_GA * GA + B_FIRST * FIRST
V_SM_SP = ASC_SM_random * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
            + B_SENIOR * SENIOR + B_FIRST * FIRST
V = {3: V_Car_SP,1: V_Train_SP,2: V_SM_SP}

# Availability
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Definition of nests
car = 1.0, [3]
rail = MU_rail, [1, 2]
nests = car, rail

# Estimation of the model
# The choice model is a logit, with availability conditions
prob = models.nested(V,av,nests,CHOICE)
logprob = log(MonteCarlo(prob))
biogeme = bio.BIOGEME(database,logprob,numberOfDraws=300)

biogeme.modelName = "Part1_MEV_Mixture"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)